# Final Project - Group 4: Revenue Dynamics in Online Event Ticket Sales

## 1. Defining the Question(s)
- Main question
- Supporting questions
- Motivation for analysis

## 2. Data Collection
- Dataset source
- Feature descriptions
- Context and background

## 3. Data Cleaning and Preparation
- Data quality issues and resolutions
- Feature engineering steps
- Handling missing values

## 4. Data Analysis
- Visualizations
- Resampling (Bootstrap, Cross-Validation)
- Statistical modeling

## 5. Interpretation and Reporting
- Key findings
- Insights and implications
- Limitations and future directions
